In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Load Iris dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
Train_df = pd.DataFrame(np.column_stack([X_train, y_train]), columns=iris.feature_names + ['target'])
print(f"Shape of X_train: {X_train.shape}")

# Selecting Initial Centroids for 3 classes (6 centroids in total)
class_0_indices = np.where(y_train == 0)[0]
class_1_indices = np.where(y_train == 1)[0]
class_2_indices = np.where(y_train == 2)[0]

class_0_centroids = Train_df.iloc[np.random.choice(class_0_indices, size=2)][iris.feature_names].values
class_1_centroids = Train_df.iloc[np.random.choice(class_1_indices, size=2)][iris.feature_names].values
class_2_centroids = Train_df.iloc[np.random.choice(class_2_indices, size=2)][iris.feature_names].values

initial_centroid_df = pd.DataFrame(np.concatenate([class_0_centroids, class_1_centroids, class_2_centroids]), columns=iris.feature_names)
initial_centroid_df['target'] = [0, 0, 1, 1, 2, 2]

initial_centroids = initial_centroid_df[iris.feature_names].values
print(f"Initial centroids shape: {initial_centroids.shape}")

# RBF Kernel
def rbf_kernel(x, c, gamma=0.1):
    return np.exp(-gamma * np.linalg.norm(x - c)**2)

# Initial Projection
def initial_projection(X, centroids, gamma=0.1):
    M = len(centroids)
    projection = np.zeros((X.shape[0], M))
    for i, x in enumerate(X):
        for j, c in enumerate(centroids):
            projection[i, j] = rbf_kernel(x, c, gamma)
    return projection

class WLS_SVC:
    def __init__(self, C=1.0):
        self.C = C
        self.alpha = None
        self.w = None
        self.b = 0
    
    def fit(self, X_proj, y):  # Ensure we use projected data here
        l, M = X_proj.shape
        self.alpha = np.zeros(l)
        self.w = np.zeros(M)  # Initialize w with same dimensionality as features
        self.b = 0

        for _ in range(10):  # A fixed number of iterations for simplicity
            for i in range(l):
                e_i = y[i] - (X_proj[i] @ self.w + self.b)
                a_i = 2 * self.alpha[i] * y[i] / (1 + y[i] * (X_proj[i] @ self.w + self.b))
                self.w += a_i * y[i] * X_proj[i]  # Update w based on projected data
                self.b += a_i * y[i]
                self.alpha[i] = min(max(self.alpha[i] + e_i, 0), self.C)
    
    def predict(self, X):
        # Ensure that the dimensions are consistent
        if X.shape[1] != self.w.shape[0]:
            raise ValueError(f"Dimension mismatch: X.shape[1]={X.shape[1]}, self.w.shape[0]={self.w.shape[0]}")
        return np.sign(X @ self.w + self.b)

# Helper function to find new centroids (support vectors)
def find_new_centroids(X, y, model, centroids, gamma=0.1):
    X_proj = initial_projection(X, centroids, gamma)  # Project X using current centroids
    print(f"X_proj shape: {X_proj.shape}")

    predictions = model.predict(X_proj)
    support_vectors = X[np.abs(y - predictions) > 0.1]  # Find support vectors
    print(f"Support vectors shape: {support_vectors.shape}")

    return support_vectors

def iterative_poker(X, y, initial_centroids, iterations=10, gamma=0.1):
    centroids = initial_centroids
    wls_svc = WLS_SVC(C=1.0)
    
    for iteration in range(iterations):
        projection = initial_projection(X, centroids, gamma)
        print(f"Iteration {iteration} projection shape: {projection.shape}")
        wls_svc.fit(projection, y)
        
        # Ensure we update centroids based on the current set of centroids
        new_centroids = find_new_centroids(X, y, wls_svc, centroids, gamma)

        if len(new_centroids) == 0:
            break  # No new centroids to add
        
        # Limit the total number of centroids to avoid dimension mismatch
        total_centroids = np.vstack([centroids, new_centroids])
        centroids = total_centroids[:initial_centroids.shape[0] + 10]
        print(f"Updated centroids shape: {centroids.shape}")

    return wls_svc, centroids

# Prepare Data for POKER
X_train_ = Train_df[iris.feature_names].values
y_train_ = Train_df['target'].values
y_train_ = 2 * y_train_ - 1  # Convert labels to -1 and 1
y_test_ = 2 * y_test - 1  # Convert labels to -1 and 1

# Apply POKER
poker_model, final_centroids = iterative_poker(X_train_, y_train_, initial_centroids)
print(f"Final centroids shape: {final_centroids.shape}")

poker_projection = initial_projection(X_test, final_centroids)  # Project test data using final centroids
print(f"Test projection shape: {poker_projection.shape}")

# Check dimensions before prediction
if poker_projection.shape[1] != poker_model.w.shape[0]:
    raise ValueError(f"Dimension mismatch: poker_projection.shape[1]={poker_projection.shape[1]}, poker_model.w.shape[0]={poker_model.w.shape[0]}")

poker_predictions = poker_model.predict(poker_projection)
poker_accuracy = accuracy_score(y_test_, poker_predictions)

# Apply regular SVM
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_, y_train_)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test_, svm_predictions)

print(f"POKER Accuracy: {poker_accuracy}")
print(f"SVM Accuracy: {svm_accuracy}")